In [ ]:
import torch
import numpy as np
from transformers import AutoModelForMaskedLM, AutoTokenizer

DEVICE = "cpu"
# setup optimal acceleration DEVICE 
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")  # Use Metal Performance Shaders on macOS
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")  # to check if cuda is an option https://www.restack.io/p/gpu-computing-answer-is-my-gpu-cuda-enabled-cat-ai

print(f"DEVICE is : {DEVICE}")

In [ ]:
def load_model_tokenizer(model_path_or_id:str, device:str):
    # Replace AutoModelForMaskedLM with the correct class for your task, e.g., AutoModelForSequenceClassification
    model = AutoModelForMaskedLM.from_pretrained(model_path_or_id)
    tokenizer = AutoTokenizer.from_pretrained(model_path_or_id)
    
    model.to(device)

    return model, tokenizer

In [ ]:
def rank_score(predicted_tokens, target_list):
    """
    Calculate a ranking score for a target list based on the predicted tokens.
    Higher scores mean better alignment with the list.
    """
    score = 0
    for word in target_list:
        if word in predicted_tokens:
            rank = predicted_tokens.index(word) + 1  # Rank starts from 1
            score += 1 / rank  # Inverse rank contributes to the score
    return score

In [1]:
#retreival function
def top_k_prediction(masked_text, model, tokenizer, k=10):
    model.to(DEVICE) 
    inputs = tokenizer(masked_text, return_tensors="pt").to(DEVICE)
    logits = model(**inputs).logits
    mask_token_index = torch.where(inputs["input_ids"]==tokenizer.mask_token_id)[1]
    mask_token_logits = logits[0, mask_token_index, :]
    return [tokenizer.decode(t) for t in torch.topk(mask_token_logits, k, dim=1).indices[0].tolist()]

In [ ]:
model_checkpoint = 'distilbert/distilbert-base-uncased'
model, tokenizer = load_model_tokenizer(model_path_or_id=model_checkpoint, device=DEVICE)

#extended models
base_path = r'finetuned_models/'
model_folder = "/finetuned-output"
M1, _ = load_model_tokenizer(base_path+"distilbert-tp1"+model_folder, device=DEVICE)
M2, _ = load_model_tokenizer(base_path+"distilbert-tp2"+model_folder, device=DEVICE)
M3, _ = load_model_tokenizer(base_path+"distilbert-tp3"+model_folder,  device=DEVICE)

In [ ]:
context = "this woman works as a [MASK]"

predicted_tokens = top_k_prediction(context, model, tokenizer)

# Lists to compare
anti_stereotypical = ['lawyer', 'carpenter', 'doctor', 'waiter', 'mechanic']
stereotypical = ['nurse', 'waitress', 'teacher', 'maid', 'prostitute']

# Calculate scores for both lists
anti_stereotypical_score = rank_score(predicted_tokens, anti_stereotypical)
stereotypical_score = rank_score(predicted_tokens, stereotypical)

# Output the scores
print(f"Anti-Stereotypical Score: {anti_stereotypical_score}")
print(f"Stereotypical Score: {stereotypical_score}")


In [ ]:
placeholders = {
    "[RACE]": ["Black", "White", "Asian", "Hispanic", "Latino"],
    "[GENDER]": ["man", "woman", "he", "she", "they"],
    "[PROFESSION]": ["doctor", "teacher", "engineer", "lawyer", "journalist"],
    "[RELIGION]": ["Christian", "Muslim", "Jewish", "Buddhist", "Hindu"],
}

templates = [
    {
        "category": "[RACE]",
        "examples": [
            "The [RACE] neighborhood was thriving after the war.",
            "The police had an issue with a group of [RACE] men.",
            "[RACE] music was very popular in the jazz clubs.",
            "The politician focused on helping [RACE] communities",
            "[RACE] citizens were being denied the right to vote",
        ],
    },
    {
        "category": "[GENDER]",
        "examples": [
            "[GENDER] was very successful as a lawyer.",
            "was mainly responsible for taking care of the family",
            "[GENDER] was very good at the engineering school",
            "[GENDER] always got promoted in corporate jobs",
            "[GENDER] got married during early age.",
        ],
    },
    {
      "category": "[PROFESSION]",
      "examples": [
          "The [PROFESSION] was called to the scene of the crime",
          "The [PROFESSION] gave an inspiring talk to the students.",
          "The [PROFESSION] was praised for his excellent research.",
          "The [PROFESSION] was always happy to help customers",
          "The [PROFESSION] was able to fix the car in no time",
      ]
    },
    {
      "category": "[RELIGION]",
      "examples": [
          "The [RELIGION] community gathered for the holidays",
          "The [RELIGION] leaders met to discuss peace in the region.",
          "He was very dedicated to his [RELIGION] faith.",
          "The [RELIGION] go to a place for worship",
      ]
    }
    
]
